# FlyMeeting Talk Figures

In [1]:
import os
import sys
from pathlib import Path
from itertools import zip_longest

from IPython.display import display, HTML, Markdown
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Project level imports
sys.path.insert(0, '../lib')
from larval_gonad.notebook import Nb
from larval_gonad.plotting import make_figs, TSNEPlot
from larval_gonad.config import memory
from larval_gonad.cell_selection import SOMA, EARLY_GERM, LATE_GERM
from larval_gonad.x_to_a import multi_chrom_boxplot, commonly_expressed, CHROMS_CHR

# Setup notebook
nbconfig = Nb.setup_notebook(subproject_dir='../output/FlyMeeting_2018_poster', seurat_dir='../output/combined_testis_force', formats=['png', 'pdf'], styles=['notebook', 'poster'])

last updated: 2018-04-10 
Git hash: 6eb6ba5ecfa67f0cc16e9f71644f0e86b7762ef2


In [2]:
colors = sns.color_palette('tab20', n_colors=12, desat=.8)
greys = sns.color_palette('Greys')
reds = sns.color_palette('Reds')
reds[0] = greys[1]
blacks = [greys[1], '#000000']

boxplot_colors = [
    reds[-1],     # X
    '#ffffff', # 2L
    '#ffffff', # 2R
    '#ffffff', # 3L
    '#ffffff', # 3R
    reds[-1],     # 4
]

In [3]:
clusters = nbconfig.seurat.get_clusters()
tsne = nbconfig.seurat.get_tsne()
norm = nbconfig.seurat.get_normalized_read_counts()

In [4]:
@make_figs(nbconfig.fig_name('tsne_clusters'), styles=['poster-full'], formats='png')
def _plot():
    TSNEPlot(data=tsne.join(clusters['res.0.4']), hue='res.0.4', palette=colors, s=100)
    ax = plt.gca()
    ax.set_aspect('equal')
    sns.despine()
    ax.legend_.remove()
    
_plot()

In [5]:
def gene_tsne(gene, ax=None, cbar=False, **kwargs):
    fbgn = nbconfig.symbol2fbgn[gene]
    if ax is None:
        fig, ax = plt.subplots(1)
    try:
        TSNEPlot(data=tsne.join(norm.T), hue=fbgn, palette=reds, ax=ax, cbar=cbar, s=30, **kwargs)
    except KeyError:
        pass

    ax.set_aspect('equal')
    ax.set_xlabel('')
    ax.set_ylabel('')
    
    if gene.startswith('CG'):
        _gene = gene
    else:
        _gene = ''
        for i, l in enumerate(gene):
            if i == 0:
                _gene += l.lower()
            else:
                _gene += l
    
    ax.set_title(f'{_gene}', style='italic', fontdict={'size': 30})
    sns.despine()
    plt.tight_layout()

In [6]:
import string
def sanitize(name):
    valid_chars = "-_.%s%s" %(string.ascii_letters, string.digits)
    return ''.join([x for x in name if x in valid_chars])

In [7]:
sharvani = [
    'tj',
    'hui',
    'glob1',
    'CG13748',
    'retn',
    'vkg',
    'bam',
    'fzo',
    'esg',
    'upd1',
    'nord',
    'bol',
    'nos',
    'Tpc2',
    'p53',
    'Tsp74F',
    'CG42458',
    'Mapmodulin',
]



for g in sharvani:
    _g = sanitize(g)
    @make_figs(nbconfig.fig_name(f'{_g}'), styles=['poster'], formats=['png'])
    def _plot(gene):
        gene_tsne(gene, edgecolor='k')
    _plot(g)
    plt.close()